In [1]:
!pip install ckanapi
from ckanapi import *
import csv
import requests

api = RemoteCKAN('https://dadosabertos.capes.gov.br')
datasets = api.action.package_list()

In [2]:
#Função para filtrar a lista de datasets, colocando os que se adequam ao filtro
def filterCatch(data, filter):
  return [str for str in data if
          any(sub in str for sub in filter)]

#Função para filtrar a lista de datasets, retirando os que se adequam ao filtro
def filterMiss(data, filter):
  return [str for str in data if
          not any(sub in str for sub in filter)]

In [5]:
import logging
import time

import os

def download_dataset(database, caminho, max_attempts=3):
    for dataset in database:
        try:
            archive = api.action.package_show(id=dataset)
            link_archive = archive['resources'][0]['url']
            name_archive = caminho + dataset + '.csv'

            # Verificar se o arquivo já existe

            for attempt in range(max_attempts):
                try:
                    print(f"Tentativa {attempt + 1}/{max_attempts}")
                    !wget -O {name_archive} --retry-connrefused {link_archive}
                    break  # Download bem-sucedido

                except ConnectionError as e:
                    logging.error(f"Erro de conexão ao baixar dataset {dataset} (tentativa {attempt + 1}/{max_attempts}): {e}")
                    time.sleep(60)  # Aguardar 1 minuto antes de tentar novamente

                    # Tentar continuar com -c em caso de erro de conexão
                    !wget -c {name_archive} {link_archive}

                except Exception as e:
                    logging.error(f"Erro inesperado ao baixar dataset {dataset}: {e}")

        except Exception as e:
            logging.error(f"Erro ao obter informações do dataset {dataset}: {e}")

In [3]:
filters=["2021-a-2024"]
database=filterCatch(datasets,filters)
print(*database,sep="\n")

2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil
2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-detalhes-da-producao-intelectual-artistica-de-programas-de-pos-graduacao-stricto-sensu
2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao
2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu
2021-a-2024-discentes-da-pos-graduacao-stricto-sensu-do-brasil
2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-financiadores-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-projetos-dos-programas-de-pos-graduacao-stricto-sensu

In [4]:
filters=["artistica"]
database=filterMiss(database,filters)
print(*database,sep="\n")

2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil
2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao
2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu
2021-a-2024-discentes-da-pos-graduacao-stricto-sensu-do-brasil
2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-financiadores-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil


In [ ]:
download_dataset(database,'csv/')

In [6]:
for dataset in database:
    print(dataset)

2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil
2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao
2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu
2021-a-2024-discentes-da-pos-graduacao-stricto-sensu-do-brasil
2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-financiadores-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil


In [8]:
#CODIGO PARA A LISTAGEM DOS DATASETS DISPONIVEIS EM CSV DAS CATEGORIAS DOS DADOS ABERTOS DO CAPES
for num in range(0, len(database)):#PARA CADA CATEGORIA
    archive = api.action.package_show(id=database[num])
    print(database[num])#SE MOSTRA QUAL ESTA NO MOMENTO
    for i in range(0, len(archive['resources'])):#E PARA CADA RECURSO, DIZ-SE POR CADA ARQUIVO
        if(archive['resources'][i]['format'] == "CSV"):#SE O TIPO FOR CSV
            print(f"{i} {archive['resources'][i]['description']}")#ENTAO MOSTRA A DESCRIÇÃO
    print('\n')
            

2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil
0 Ano Base - 2021 a 2022, Artística Subtipo Artes Cênicas
2 Ano Base - 2021 a 2022, Artística Subtipo Artes Visuais
4 Ano Base - 2021 a 2022, Artística Subtipo Música
6 Ano Base - 2021 a 2022, Artística Subtipo Outros
8 Ano Base - 2021, Bibliográfica Subtipo Trabalho em Anais
10 Ano Base - 2022, Bibliográfica Subtipo Trabalho em Anais
12 Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista
14 Ano Base - 2021, Bibliográfica Subtipo Artigo em Periódico
17 Ano Base - 2022, Bibliográfica Subtipo Artigo em Periódico
20 Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro
22 Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro
24 Ano Base - 2021 a 2022, Bibliográfica Subtipo Partitura Musical
26 Ano Base - 2021 a 2022, Bibliográfica Subtipo Tradução
28 Ano Base - 2021 a 2022, Técnica Subtipo Apresentação de Trabalho
30 Ano Base - 2021 a 2022, Técnica Subtipo Cartas, Mapas ou Simi

'text/csv'